In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



from sklearn.feature_selection import RFE
import statsmodels.formula.api as smf

In [3]:
#import cleaned
df = pd.read_csv('cleaned_data.csv')

In [4]:
df.corr()["cast_total_facebook_likes"]

num_critic_for_reviews       0.279639
duration                     0.143359
director_facebook_likes      0.145200
actor_3_facebook_likes       0.551871
actor_1_facebook_likes       0.932273
gross                        0.266763
num_voted_users              0.288863
cast_total_facebook_likes    1.000000
facenumber_in_poster         0.085601
num_user_for_reviews         0.216289
budget                       0.027778
title_year                   0.152753
actor_2_facebook_likes       0.706831
imdb_score                   0.128256
aspect_ratio                 0.101322
movie_facebook_likes         0.241069
profit                       0.049964
action                       0.033454
adventure                    0.040156
animation                   -0.003015
biography                    0.012525
comedy                      -0.035042
crime                        0.015020
documentary                 -0.040534
drama                       -0.002791
family                      -0.000840
fantasy     

We can see from above that facebook likes is extremely correlated with the number of facebook likes of actor 1 (0.932273), as well as actor 2 (0.706831), and actor 3(0.551871). This should be taken into account when making the model (interactions).

## Which combination of actors are most profitable?

In [5]:
df_people = df[["director_name", "director_facebook_likes",  "actor_1_name", "actor_1_facebook_likes","actor_2_name", "actor_2_facebook_likes", "actor_3_name","actor_3_facebook_likes", "cast_total_facebook_likes",  "profit"]]

In [6]:
yay = df_people.groupby(["director_name","actor_1_name", "actor_2_name", "actor_3_name"])[["profit"]].agg("mean").sort_values(by = "profit", ascending=False)
yay

,,,,profit
director_name,actor_1_name,actor_2_name,actor_3_name,
James Cameron,CCH Pounder,Joel David Moore,Wes Studi,5.235058e+08
Colin Trevorrow,Bryce Dallas Howard,Judy Greer,Omar Sy,5.021773e+08
James Cameron,Leonardo DiCaprio,Kate Winslet,Gloria Stuart,4.586723e+08
George Lucas,Harrison Ford,Peter Cushing,Kenny Baker,4.499357e+08
Steven Spielberg,Henry Thomas,Dee Wallace,Peter Coyote,4.244495e+08
...,...,...,...,...
Katsuhiro Ôtomo,William Hootkins,Robin Atkin Downes,Rosalind Ayres,-2.127110e+09
Hayao Miyazaki,Minnie Driver,Jada Pinkett Smith,Billy Crudup,-2.397702e+09
Lajos Koltai,Marcell Nagy,Péter Fancsikai,Bálint Péntek,-2.499804e+09


In [7]:
yay[:3]

,,,,profit
director_name,actor_1_name,actor_2_name,actor_3_name,
James Cameron,CCH Pounder,Joel David Moore,Wes Studi,523505847.0
Colin Trevorrow,Bryce Dallas Howard,Judy Greer,Omar Sy,502177271.0
James Cameron,Leonardo DiCaprio,Kate Winslet,Gloria Stuart,458672302.0


In [8]:
df_people.groupby(["director_name","actor_1_name", "actor_2_name", "actor_3_name"]).agg("count")

,,,,director_facebook_likes,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,cast_total_facebook_likes,profit
director_name,actor_1_name,actor_2_name,actor_3_name,,,,,,
Aaron Schneider,Bill Murray,Robert Duvall,Bill Cobbs,2,2,2,2,2,2
Aaron Seltzer,Alyson Hannigan,Carmen Electra,Fred Willard,2,2,2,2,2,2
Abel Ferrara,Isabella Rossellini,Vincent Gallo,Gretchen Mol,2,2,2,2,2,2
Adam Goldberg,Judy Greer,Marisa Coughlan,Nicky Katt,2,2,2,2,2,2
Adam Marcus,Kane Hodder,Leslie Jordan,Erin Gray,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...
Zack Snyder,Ty Burrell,Kevin Zegers,Mekhi Phifer,6,6,6,6,6,6
Zak Penn,Zak Penn,Gabriel Beristain,John Bailey,3,3,3,3,3,3
Zal Batmanglij,Alexander Skarsgård,Julia Ormond,Jason Ritter,2,2,2,2,2,2


Most profitable combination of director and actors is James Cameron, CCH Pounder, Joel David Moore, and Wes Studi.

In [9]:
df_people.groupby(["director_name"])[["profit"]].agg("mean").sort_values(by = "profit", ascending=False).head(10)


,profit
director_name,
Tim Miller,3.050243e+08
Colin Trevorrow,2.883546e+08
George Lucas,2.859128e+08
Richard Marquand,2.766254e+08
Kyle Balda,2.620296e+08
Chris Buck,2.507366e+08
Yarrow Cheney,2.485055e+08
Pierre Coffin,2.451005e+08
Joss Whedon,2.298458e+08


In [10]:
df_people.groupby(["actor_1_name"])[["profit"]].agg("mean").sort_values(by = "profit", ascending=False).head(10)


,profit
actor_1_name,
Wayne Knight,2.937840e+08
Rupert Everett,2.864710e+08
Henry Thomas,2.823595e+08
Catherine Dyer,2.269504e+08
Josh Gad,2.130122e+08
Kathleen Freeman,2.076520e+08
Hattie McDaniel,1.946783e+08
Adriana Caselotti,1.829255e+08
Olivia Newton-John,1.753600e+08


In [11]:
df_people.groupby(["actor_2_name"])[["profit"]].agg("mean").sort_values(by = "profit", ascending=False).head(10)


,profit
actor_2_name,
Peter Cushing,4.499357e+08
Robert Downey Jr.,3.385169e+08
Ed Skrein,3.050243e+08
Leonard Roberts,2.913236e+08
Jennifer Saunders,2.864710e+08
Ian McDiarmid,2.766254e+08
Kenny Baker,2.721588e+08
Miranda Cosgrove,2.451005e+08
Joel David Moore,2.350092e+08


In [12]:
df_people.groupby(["actor_3_name"])[["profit"]].agg("mean").sort_values(by = "profit", ascending=False).head(10)


,profit
actor_3_name,
Omar Sy,502177271.0
Gloria Stuart,458672302.0
Niketa Calame,377783777.0
Ian McDiarmid,359544677.0
Anthony Reynolds,329999255.0
Stefan Kapicic,305024263.0
Bob Peck,293784000.0
Keir O'Donnell,291323553.0
Brad Garrett,286838870.0


In [13]:
#bin profits into 3 bins, with each actor profit in each bin
df_people["profit_bin"] = pd.cut(df_people["profit"], bins=3, labels=["low", "medium", "high"])

df_people.head()

/var/folders/9x/2dgcd7cn3894j3wpmddp46940000gn/T/ipykernel_23927/1858527934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_people["profit_bin"] = pd.cut(df_people["profit"], bins=3, labels=["low", "medium", "high"])


,director_name,director_facebook_likes,actor_1_name,actor_1_facebook_likes,actor_2_name,actor_2_facebook_likes,actor_3_name,actor_3_facebook_likes,cast_total_facebook_likes,profit,profit_bin
0,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,4834,523505847.0,high
1,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,4834,523505847.0,high
2,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,4834,523505847.0,high
3,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,4834,523505847.0,high
4,Gore Verbinski,563.0,Johnny Depp,40000.0,Orlando Bloom,5000.0,Jack Davenport,1000.0,48350,9404152.0,high
